In [2]:
import duckdb

In [3]:
DATABASE = "../database.db"

In [5]:
with duckdb.connect(DATABASE) as con:
    con.sql("""
    SELECT *
    FROM player
    LIMIT 5
    ;
    """).show()

┌───────────┬──────────────────┬──────────┬──────────────┬───────────────┐
│ player_id │       name       │ position │ current_team │ jersey_number │
│  varchar  │     varchar      │ varchar  │   varchar    │    double     │
├───────────┼──────────────────┼──────────┼──────────────┼───────────────┤
│ 6462      │ Ellis Richardson │ TE       │ NULL         │          45.0 │
│ 11255     │ Nick Amoah       │ OL       │ NULL         │           0.0 │
│ 8842      │ Malkelm Morrison │ CB       │ NULL         │           0.0 │
│ 7926      │ Carl Tucker      │ TE       │ NULL         │           0.0 │
│ 1875      │ C.J. Mosley      │ LB       │ NULL         │          57.0 │
└───────────┴──────────────────┴──────────┴──────────────┴───────────────┘

